<a href="https://colab.research.google.com/github/froggagul/SimpleCNN/blob/master/cnn-inception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from __future__ import print_function
import argparse
import torch
from torch import nn, optim, cuda
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.autograd import Variable
import time
device = 'cuda' if cuda.is_available() else 'cpu'
print(f'Training MNIST Model on {device}\n{"=" * 44}')


Training MNIST Model on cuda


In [3]:
batch_size = 64

train_dataset = datasets.MNIST(root='./data',
                              train=True,
                              transform=transforms.ToTensor(),
                              download=True)
test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [10]:
class InceptionA(nn.Module):
  def __init__(self, in_channels):
    super(InceptionA, self).__init__()
    self.branch1x1 = nn.Conv2d(in_channels, 16, kernel_size=1)
    self.branch_pool_1 = nn.AvgPool2d(kernel_size=3, stride=1, padding=1)
    self.branch_pool_2 = nn.Conv2d(in_channels, 24, kernel_size=1)

    self.branch5x5_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
    self.branch5x5_2 = nn.Conv2d(16, 24, kernel_size=5, padding=2)

    self.branch3x3dbl_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
    self.branch3x3dbl_2 = nn.Conv2d(16, 24, kernel_size=3, padding=1)
    self.branch3x3dbl_3 = nn.Conv2d(24, 24, kernel_size=3, padding=1)

  def forward(self, x):
    branch1x1 = self.branch1x1(x)
    
    branch_pool = self.branch_pool_1(x)
    branch_pool = self.branch_pool_2(branch_pool)

    branch5x5 = self.branch5x5_1(x)
    branch5x5 = self.branch5x5_2(branch5x5)

    branch3x3dbl = self.branch3x3dbl_1(x)
    branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
    branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)

    outputs = [branch1x1, branch5x5, branch3x3dbl, branch_pool]
    return torch.cat(outputs, 1)

class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
    # 24 + 24 + 24 + 16 = 88
    self.conv2 = nn.Conv2d(88, 20, kernel_size=5)

    self.incept1 = InceptionA(in_channels=10)
    self.incept2 = InceptionA(in_channels=20)

    self.mp = nn.MaxPool2d(2)
    self.fc = nn.Linear(1408, 10)

  def forward(self, x):
    in_size = x.size(0)
    x = F.relu(self.mp(self.conv1(x)))
    x = self.incept1(x)
    x = F.relu(self.mp(self.conv2(x)))
    x = self.incept2(x)
    x = x.view(in_size, -1)  # flatten the tensor
    x = self.fc(x)
    return F.log_softmax(x)

model = Net()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad() # 기울기 초기화
        output = model(data)
        loss = criterion(output, target)
        loss.backward()  # back propagation, 역전파
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} | Batch Status: {}/{} ({:.0f}%) | Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).item()
        # get the index of the max
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(f'===========================\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)')


if __name__ == '__main__':
    since = time.time()
    for epoch in range(1, 10):
        epoch_start = time.time()
        train(epoch)
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Training time: {m:.0f}m {s:.0f}s')
        test()
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Testing time: {m:.0f}m {s:.0f}s')

    m, s = divmod(time.time() - since, 60)
    print(f'Total Time: {m:.0f}m {s:.0f}s\nModel was trained on {device}!')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 | Batch Status: 0/60000 (0%) | Loss: 2.300445
Train Epoch: 1 | Batch Status: 640/60000 (1%) | Loss: 2.294341
Train Epoch: 1 | Batch Status: 1280/60000 (2%) | Loss: 2.303401
Train Epoch: 1 | Batch Status: 1920/60000 (3%) | Loss: 2.300732
Train Epoch: 1 | Batch Status: 2560/60000 (4%) | Loss: 2.296707
Train Epoch: 1 | Batch Status: 3200/60000 (5%) | Loss: 2.287135
Train Epoch: 1 | Batch Status: 3840/60000 (6%) | Loss: 2.304753
Train Epoch: 1 | Batch Status: 4480/60000 (7%) | Loss: 2.300517
Train Epoch: 1 | Batch Status: 5120/60000 (9%) | Loss: 2.301292
Train Epoch: 1 | Batch Status: 5760/60000 (10%) | Loss: 2.295539
Train Epoch: 1 | Batch Status: 6400/60000 (11%) | Loss: 2.274105
Train Epoch: 1 | Batch Status: 7040/60000 (12%) | Loss: 2.286982
Train Epoch: 1 | Batch Status: 7680/60000 (13%) | Loss: 2.272845
Train Epoch: 1 | Batch Status: 8320/60000 (14%) | Loss: 2.277978
Train Epoch: 1 | Batch Status: 8960/60000 (15%) | Loss: 2.264549
Train Epoch: 1 | Batch Status: 9600/60